In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
import re

In [0]:
train = pd.read_excel("/content/drive/My Drive/Colab Notebooks/Predicting Food Delivery Time/Data_Train.xlsx")
test = pd.read_excel("/content/drive/My Drive/Colab Notebooks/Predicting Food Delivery Time/Data_Test.xlsx")

In [72]:
train.head()

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time
0,ID_6321,"FTI College, Law College Road, Pune","Fast Food, Rolls, Burger, Salad, Wraps",₹200,₹50,3.5,12,4,30 minutes
1,ID_2882,"Sector 3, Marathalli","Ice Cream, Desserts",₹100,₹50,3.5,11,4,30 minutes
2,ID_1595,Mumbai Central,"Italian, Street Food, Fast Food",₹150,₹50,3.6,99,30,65 minutes
3,ID_5929,"Sector 1, Noida","Mughlai, North Indian, Chinese",₹250,₹99,3.7,176,95,30 minutes
4,ID_6123,"Rmz Centennial, I Gate, Whitefield","Cafe, Beverages",₹200,₹99,3.2,521,235,65 minutes


In [73]:
train.isnull().sum()

Restaurant       0
Location         0
Cuisines         0
Average_Cost     0
Minimum_Order    0
Rating           0
Votes            0
Reviews          0
Delivery_Time    0
dtype: int64

In [74]:
train.describe(include = 'all')

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time
count,11094,11094,11094,11094,11094,11094,11094,11094,11094
unique,7480,35,2179,26,18,33,1103,761,7
top,ID_7184,"Mico Layout, Stage 2, BTM Layout,Bangalore",North Indian,₹200,₹50,-,-,-,30 minutes
freq,22,947,850,3241,10118,1191,2074,2312,7406


In [75]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11094 entries, 0 to 11093
Data columns (total 9 columns):
Restaurant       11094 non-null object
Location         11094 non-null object
Cuisines         11094 non-null object
Average_Cost     11094 non-null object
Minimum_Order    11094 non-null object
Rating           11094 non-null object
Votes            11094 non-null object
Reviews          11094 non-null object
Delivery_Time    11094 non-null object
dtypes: object(9)
memory usage: 780.2+ KB


In [76]:
test.head()

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews
0,ID_2842,"Mico Layout, Stage 2, BTM Layout,Bangalore","North Indian, Chinese, Assamese",₹350,₹50,4.2,361,225
1,ID_730,"Mico Layout, Stage 2, BTM Layout,Bangalore","Biryani, Kebab",₹100,₹50,NEW,-,-
2,ID_4620,"Sector 1, Noida",Fast Food,₹100,₹50,3.6,36,16
3,ID_5470,"Babarpur, New Delhi, Delhi","Mithai, North Indian, Chinese, Fast Food, Sout...",₹200,₹50,3.6,66,33
4,ID_3249,"Sector 1, Noida","Chinese, Fast Food",₹150,₹50,2.9,38,14


In [77]:
test.isnull().sum()

Restaurant       0
Location         0
Cuisines         0
Average_Cost     0
Minimum_Order    0
Rating           0
Votes            0
Reviews          0
dtype: int64

In [78]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2774 entries, 0 to 2773
Data columns (total 8 columns):
Restaurant       2774 non-null object
Location         2774 non-null object
Cuisines         2774 non-null object
Average_Cost     2774 non-null object
Minimum_Order    2774 non-null object
Rating           2774 non-null object
Votes            2774 non-null object
Reviews          2774 non-null object
dtypes: object(8)
memory usage: 173.5+ KB


In [0]:
train['Average_Cost'].replace('for', '200', inplace = True)
train['Avg_Cost_Cleaned'] = train['Average_Cost'].apply(lambda x: int(re.sub("[^0-9]", "", x)))
train['Min_Order_Cleaned'] = train['Minimum_Order'].apply(lambda x: int(re.sub("[^0-9]", "", x)))

test['Avg_Cost_Cleaned'] = test['Average_Cost'].apply(lambda x: int(re.sub("[^0-9]", "", x)))
test['Min_Order_Cleaned'] = test['Minimum_Order'].apply(lambda x: int(re.sub("[^0-9]", "", x)))

In [0]:
def max_features_in_single_row(train, test, delimiter):
  max_info = 0 
  item_lis = list(train.append(test))
  for i in item_lis:
    if len(i.split("{}".format(delimiter))) > max_info:
      max_info = len(i.split("{}".format(delimiter)))
  return max_info

In [0]:
def feature_splitter(feature, col_name, delimiter, max_info):
  item_lis = list(feature)
  extracted_features = {}

  for i in range(max_info):
    extracted_features['{}_Feature_{}'.format(col_name, i+1)] = []

  return extracted_features

In [0]:
loc_max = max_features(test['Location'],train['Location'], ',')
train_Location_splits = feature_splitter(train['Location'], 'Location', ',', loc_max)
test_Location_splits = feature_splitter(test['Location'], 'Location', ',', loc_max)

In [0]:
cus_max = max_features(test['Cuisines'],train['Cuisines'], ',')
train_Cuisines_splits = feature_splitter(train['Cuisines'], 'Cuisines', ',', cus_max)
test_Cuisines_splits = feature_splitter(test['Cuisines'], 'Cuisines', ',', cus_max)

In [0]:
def non_numeric_values(series):
  non_numeric = []
  for i in series.unique():
    try :
      i = float(i)
    except:
      non_numeric.append(i)
  return non_numeric

def replace_non_numeric_values(series, type_, fill_with = None, method = 'mean'):  
  non_numeric = non_numeric_values(series)
  
  series = series.replace(non_numeric, np.nan, inplace = False)
  finding_mean = series.astype(float).mean()
  series.fillna(finding_mean, inplace = True)
  
  series = series.astype(type_)
  return series

In [0]:
train['Rating_Cleaned'] = replace_non_numeric_values(train['Rating'], float)
train['Votes_Cleaned'] = replace_non_numeric_values(train['Votes'], float)
train['Reviews_Cleaned'] = replace_non_numeric_values(train['Reviews'], float)

test['Rating_Cleaned'] = replace_non_numeric_values(test['Rating'], float)
test['Votes_Cleaned'] = replace_non_numeric_values(test['Votes'], float)
test['Reviews_Cleaned'] = replace_non_numeric_values(test['Reviews'], float)

In [0]:
cols = ['Restaurant',  'Avg_Cost_Cleaned','Min_Order_Cleaned', 'Rating_Cleaned', 'Votes_Cleaned',
       'Reviews_Cleaned','Delivery_Time' ]
train_new = train[cols]
test_new = test[cols[:-1]]

In [0]:
train_new = pd.concat([pd.DataFrame(train_Location_splits), pd.DataFrame(train_Cuisines_splits),train_new],sort=False,axis = 1)
test_new = pd.concat([pd.DataFrame(test_Location_splits), pd.DataFrame(test_Cuisines_splits), test_new],sort=False,axis = 1)

In [88]:
train_new.head()

,Location_Feature_1,Location_Feature_2,Location_Feature_3,Location_Feature_4,Cuisines_Feature_1,Cuisines_Feature_2,Cuisines_Feature_3,Cuisines_Feature_4,Cuisines_Feature_5,Cuisines_Feature_6,Cuisines_Feature_7,Cuisines_Feature_8,Restaurant,Avg_Cost_Cleaned,Min_Order_Cleaned,Rating_Cleaned,Votes_Cleaned,Reviews_Cleaned,Delivery_Time
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ID_6321,200,50,3.5,12.0,4.0,30 minutes
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ID_2882,100,50,3.5,11.0,4.0,30 minutes
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ID_1595,150,50,3.6,99.0,30.0,65 minutes
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ID_5929,250,99,3.7,176.0,95.0,30 minutes
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ID_6123,200,99,3.2,521.0,235.0,65 minutes


In [104]:
train_new.isnull().sum()

Location_Feature_1    0
Location_Feature_2    0
Location_Feature_3    0
Location_Feature_4    0
Cuisines_Feature_1    0
Cuisines_Feature_2    0
Cuisines_Feature_3    0
Cuisines_Feature_4    0
Cuisines_Feature_5    0
Cuisines_Feature_6    0
Cuisines_Feature_7    0
Cuisines_Feature_8    0
Restaurant            0
Avg_Cost_Cleaned      0
Min_Order_Cleaned     0
Rating_Cleaned        0
Votes_Cleaned         0
Reviews_Cleaned       0
Delivery_Time         0
dtype: int64

In [0]:
train_new.fillna('NAN', inplace = True)

In [106]:
train_new.isnull().sum()

Location_Feature_1    0
Location_Feature_2    0
Location_Feature_3    0
Location_Feature_4    0
Cuisines_Feature_1    0
Cuisines_Feature_2    0
Cuisines_Feature_3    0
Cuisines_Feature_4    0
Cuisines_Feature_5    0
Cuisines_Feature_6    0
Cuisines_Feature_7    0
Cuisines_Feature_8    0
Restaurant            0
Avg_Cost_Cleaned      0
Min_Order_Cleaned     0
Rating_Cleaned        0
Votes_Cleaned         0
Reviews_Cleaned       0
Delivery_Time         0
dtype: int64

In [107]:
test_new.fillna('NAN', inplace = True)
test_new.isnull().sum()

Location_Feature_1    0
Location_Feature_2    0
Location_Feature_3    0
Location_Feature_4    0
Cuisines_Feature_1    0
Cuisines_Feature_2    0
Cuisines_Feature_3    0
Cuisines_Feature_4    0
Cuisines_Feature_5    0
Cuisines_Feature_6    0
Cuisines_Feature_7    0
Cuisines_Feature_8    0
Restaurant            0
Avg_Cost_Cleaned      0
Min_Order_Cleaned     0
Rating_Cleaned        0
Votes_Cleaned         0
Reviews_Cleaned       0
dtype: int64

In [0]:
temp1 = []
for i in train_Cuisines_splits.keys():
  for j in train_Cuisines_splits.get(i):
    temp1.append(j)

temp2 = []
for i in test_Cuisines_splits.keys():
  for j in test_Cuisines_splits.get(i):
    temp2.append(j)

temp1.extend(temp2)

unique_cuisines = list(pd.Series(temp1).unique())
unique_cuisines.append('NAN')

In [0]:
temp1 = []
for i in train_Location_splits.keys():
  for j in train_Location_splits.get(i):
    temp1.append(j)

temp2 = []
for i in test_Location_splits.keys():
  for j in test_Location_splits.get(i):
    temp2.append(j)

temp1.extend(temp2)

unique_locations = list(pd.Series(temp1).unique())
unique_locations.append('NAN')

In [0]:
#encoding the categorical Features
from sklearn.preprocessing import LabelEncoder
le_c = LabelEncoder().fit(unique_cuisines)
le_l = LabelEncoder().fit(unique_locations)

In [0]:
for i in train_Location_splits.keys():
  train_new[i] = le_l.transform(train_new[i])
for i in train_Cuisines_splits.keys():
  train_new[i] = le_c.transform(train_new[i])

In [0]:
for i in test_Location_splits.keys():
  test_new[i] = le_l.transform(test_new[i])
for i in test_Cuisines_splits.keys():
  test_new[i] = le_c.transform(test_new[i])

In [0]:
t1 = list(train_new['Restaurant'])
t2 = list(test_new['Restaurant'])

t1.extend(t2)
unique_ids = list(set(t1))

In [0]:
le_id = LabelEncoder().fit(unique_ids)

train_new['Restaurant'] = le_id.transform(train_new['Restaurant'])
test_new['Restaurant'] = le_id.transform(test_new['Restaurant'])

In [0]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [0]:
cols = list(train_new.columns)
train_new[cols[:-1]] = sc.fit_transform(train_new[cols[:-1]])
test_new[cols[:-1]] = sc.fit_transform(test_new[cols[:-1]])

In [103]:
train_new.head()

,Location_Feature_1,Location_Feature_2,Location_Feature_3,Location_Feature_4,Cuisines_Feature_1,Cuisines_Feature_2,Cuisines_Feature_3,Cuisines_Feature_4,Cuisines_Feature_5,Cuisines_Feature_6,Cuisines_Feature_7,Cuisines_Feature_8,Restaurant,Avg_Cost_Cleaned,Min_Order_Cleaned,Rating_Cleaned,Votes_Cleaned,Reviews_Cleaned,Delivery_Time
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.625956,-0.020864,-0.180293,-0.301852,-0.464626,-0.417526,30 minutes
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.897145,-0.791117,-0.180293,-0.301852,-0.466624,-0.417526,30 minutes
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.467013,-0.405990,-0.180293,-0.035808,-0.290799,-0.326492,65 minutes
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.451808,0.364263,2.461158,0.230235,-0.136953,-0.098905,30 minutes
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.538284,-0.020864,2.461158,-1.099981,0.552361,0.391281,65 minutes


In [0]:
# importing alll the necessary packages to use the various classification algorithms

from sklearn.linear_model import LogisticRegression  # for Logistic Regression algorithm
from sklearn.model_selection import train_test_split #to split the dataset for training and testing
from sklearn.neighbors import KNeighborsClassifier  # for K nearest neighbours
from sklearn import svm  #for Support Vector Machine (SVM) Algorithm
from sklearn import metrics #for checking the model accuracy
from sklearn.tree import DecisionTreeClassifier #for using Decision Tree Algoithm

In [0]:
train, val = train_test_split(train_new, test_size = 0.3)

In [0]:
X_train = train[cols[:-1]]
Y_train = train[cols[-1]]

X_Val = val[cols[:-1]]
Y_Val = val[cols[-1]]

X_test = test_new[cols[:-1]]

In [115]:
# SVM 
model = svm.SVC()
model.fit(X_train,Y_train)

pred = model.predict(X_Val)
print('The accuracy of the SVM is:',metrics.accuracy_score(pred,Y_Val))

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


The accuracy of the SVM is: 0.7062180835085611


In [116]:
# Logistic Regression
model = LogisticRegression()
model.fit(X_train,Y_train)

pred = model.predict(X_Val)
print('The accuracy of Logistic Regression is:',metrics.accuracy_score(pred,Y_Val))

The accuracy of Logistic Regression is: 0.6939020726945029


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [122]:
# Decision Tree
model = DecisionTreeClassifier()
model.fit(X_train,Y_train)

pred = model.predict(X_Val)
print('The acccuracy of DecisionTreeClassifier is :',metrics.accuracy_score(pred,Y_Val))

predictions = model.predict(X_test)
print(predictions)

pd.DataFrame(predictions, columns = ['Delivery_Time']).to_excel("/content/drive/My Drive/Colab Notebooks/Predicting Food Delivery Time/Submission_1.xlsx", index = False)

The acccuracy of DecisionTreeClassifier is : 0.7443676779813758
['30 minutes' '30 minutes' '30 minutes' ... '45 minutes' '65 minutes'
 '45 minutes']


In [119]:
# Let's check the accuracy for various values of n for K-Nearest nerighbour
for i in range(1,11):
    model = KNeighborsClassifier(n_neighbors=i)
    model.fit(X_train,Y_train)
    pred = model.predict(X_Val)
    print('The accuracy of KNeighborsClassifier with {} neighbors is: {}'.format(i,metrics.accuracy_score(pred,Y_Val)))

The accuracy of KNeighborsClassifier with 1 neighbors is: 0.7380594773205167
The accuracy of KNeighborsClassifier with 2 neighbors is: 0.7260438570141183
The accuracy of KNeighborsClassifier with 3 neighbors is: 0.7035145689396215
The accuracy of KNeighborsClassifier with 4 neighbors is: 0.7056173024932412
The accuracy of KNeighborsClassifier with 5 neighbors is: 0.7047161309702613
The accuracy of KNeighborsClassifier with 6 neighbors is: 0.7032141784319615
The accuracy of KNeighborsClassifier with 7 neighbors is: 0.7029137879243016
The accuracy of KNeighborsClassifier with 8 neighbors is: 0.7011114448783419
The accuracy of KNeighborsClassifier with 9 neighbors is: 0.7014118353860018
The accuracy of KNeighborsClassifier with 10 neighbors is: 0.7056173024932412
